# Running Oktoberfest

## 1- Import necessary python packages

In [ ]:
import os
from oktoberfest.runner import run_job
import json
import urllib.request
import shutil
from tqdm import tqdm

## 2- Download files from zenodo required to run different tasks

### A- Get the current directory and set the file name

In [ ]:
download_dir = os.getcwd()
download_file = os.path.join(download_dir, 'Oktoberfest_input.zip')
url = 'https://zenodo.org/record/7613029/files/Oktoberfest_input.zip'

download = True  # set this to false if you already have the file and don't want to download again in the next step

### B- Download and extract files from zenodo to the same directory

In [ ]:
if download:
    with tqdm(unit="B", total=2739196307, unit_scale=True, unit_divisor=1024, miniters=1, desc=url.split("/")[-1]) as t:
        urllib.request.urlretrieve(url=url, filename=download_file, reporthook=lambda blocks, block_size, _: t.update(blocks * block_size - t.n))
    shutil.unpack_archive(download_file, download_dir)

### C- Check downloaded files

In [ ]:
input_dir = download_file[:-4]
print(f'Downloaded data is stored in {input_dir}\nContents:')
os.listdir(input_dir)

## 3- Running Different Tasks

### A- Spectral Library Generation

#### Generate config file

In [ ]:
task_config_spectral_lib = {
    "type": "SpectralLibraryGeneration",
    "tag": "",
    "inputs": {
            "search_results": input_dir + "/msms.txt",
            "search_results_type": "Maxquant",
            "library_input": input_dir + "/peptides_spectral_library.csv",
            "library_input_type": "peptides"
        },
    "output": "./out",
    "models":{
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "outputFormat": "spectronaut",
    "prediction_server": "koina.proteomicsdb.org:443",
    "ssl": True,
}

#### Save config as json

In [ ]:
with open('./spectral_library_config.json', 'w') as fp:
    json.dump(task_config_spectral_lib, fp)

#### Run spectral library generation job

In [ ]:
run_job("./spectral_library_config.json")

### B- CE Calibration

#### Generate config file

In [ ]:
task_config_ce_calibration = {
    "type": "CollisionEnergyCalibration",
    "tag": "",
    "inputs":{
        "search_results": input_dir + "/msms.txt",
        "search_type": "Maxquant",
        "spectra": input_dir,
        "spectra_type": "raw"
    },
    "output": "./out",
    "models": {
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "prediction_server": "koina.proteomicsdb.org:443",
    "ssl": True,
    "thermoExe": "ThermoRawFileParser.exe",
    "massTolerance": 20,
    "unitMassTolerance": "ppm",
    "numThreads": 4
}

#### Save config as json

In [ ]:
with open('./ce_calibration_config.json', 'w') as fp:
    json.dump(task_config_ce_calibration, fp)

#### Run ce calibration job

In [ ]:
run_job("./ce_calibration_config.json")

### C- Rescoring

#### Generate config file

In [ ]:
task_config_rescoring = {
    "type": "Rescoring",
    "tag": "",
    "inputs":{
        "search_results": input_dir + "/msms.txt",
        "search_results_type": "Maxquant",
        "spectra": input_dir,
        "spectra_type": "raw"
    },
    "output": "./out",
    "models": {
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "prediction_server": "koina.proteomicsdb.org:443",
    "ssl": True,
    "thermoExe": "ThermoRawFileParser.exe",
    "numThreads": 4,
    "fdr_estimation_method": "percolator",
    "regressionMethod": "spline",
    "allFeatures": False,
    "massTolerance": 20,
    "unitMassTolerance": "ppm"
}

#### Save config as json

In [ ]:
with open('./rescoring_config.json', 'w') as fp:
    json.dump(task_config_rescoring, fp)

#### Run rescoring job

In [ ]:
run_job("rescoring_config.json")